In [159]:
import random
import math
import numpy as np
import copy

In [545]:
mu = 50
sigma = 20
n = 2000
b = 0.1
beta = 0.1
eps=1
D = np.random.normal(mu,sigma,n)
D.sort()
width = D[n-1]-D[0]
intD = discretize(D,b)

In [546]:
def discretize(D, b):    
    n = len(D)
    discreteD = np.zeros(n)
    for i in range(n):
        discreteD[i] = int(D[i]/b)
    return discreteD

def shiftD(D,x):
    #shift towards left
    return D-x
    
def count(D, a, b):
    counter = 0
    n = len(D)
    for i in range(n):
        if D[i] >=a and D[i]<=b:
            counter+=1
    return counter

def clip(D, a, b):
    clipped = copy.deepcopy(D)
    clipped[clipped<a] = a
    clipped[clipped>b] = b
    return clipped

def LapNoise():
    a = random.uniform(0,1)
    b = math.log(1/(1-a))
    c = random.uniform(0,1)
    if c>0.5:
        return b
    else:
        return -b
    
def finiteDomainQuantile(eps, beta, tau, a,b, D):
    #[a,b] denotes range
    n = len(D)
    u = np.zeros(n+2)#utility score
    l = np.zeros(n+3)#score changing point
    l[0] = a
    for i in range(n+2):
        if i<=n/2:
            u[i] = -n/2-1+i
            l[i+1]= D[i]
        u[int(n/2+1)] = 0
        l[int(n/2+2)] = D[int(n/2)]
        if i>n/2+1:
            u[i] = n/2+1-i
            l[i]= D[i-2]
            
    l[n+2] = b
    weight = np.zeros(n+2)
    for i in range(n+2):
        weight[i] = (l[i+1]-l[i])*math.pow(np.e, eps*u[i]/2)
    weight[int(n/2+1)] = math.pow(np.e, 0)
    
    totalWeight = sum(weight)
    weight = weight/totalWeight

    i = np.random.choice(list(range(n+2)), p=weight)
    if i==int(n/2+1):
        return l[i]
    else:
        x = np.random.randint(l[i], l[i+1])
        return x

In [547]:
def unboundedDomainRadius(eps, beta, D):
    TNoise = 2/eps*LapNoise()
    T = len(D)
    i=1
    while True:
        Ti = T-4/eps*np.log(2*i**2 * np.pi**2/(3*beta)) - 2/eps*np.log(4/beta)+TNoise
        if i==1:
            Qi = count(abs(D),0,0)+4/eps*LapNoise()
        else:
            Qi = count(abs(D),0, math.pow(2, i-2))+4/eps*LapNoise()

        if Qi>Ti:
            break
        else:
            i+=1
            
    if i==1:
        return 0
    else:
        return math.pow(2,i-1)

In [548]:
def unboundedDomainShiftedMean(eps, beta, b, D):
    n = len(D)
    intD = discretize(D,b)
    rad = unboundedDomainRadius(eps/4, beta/4, intD)
    clipped = clip(intD,-rad,rad)
    interiorPoint = finiteDomainQuantile(eps, beta, n/2, -rad, rad, intD)
    shifted = clipped - interiorPoint
    width = unboundedDomainRadius(eps/4, beta/4, shifted)

    DforMean = clip(D, interiorPoint*b -width*b-2*b, interiorPoint*b +width*b+2*b)
    noisyMu = sum(DforMean)/n+8*(width*b+b)/(eps*n)*LapNoise()
    error = abs(noisyMu-sum(D)/n)
    #print("ratio is "+ str(8*(width*b+b)*np.log(4/beta)/(eps*n)/error))
    indicator = 0
    if sum(D)/n<noisyMu +8*(width*b+b)*np.log(4/beta)/(eps*n) and sum(D)/n>noisyMu - 8*(width*b+b)*np.log(4/beta)/(eps*n):
        indicator = 1
    #return [noisyMu - 8*(width*b+b)*np.log(4/beta)/(eps*n),noisyMu + 8*(width*b+b)*np.log(4/beta)/(eps*n) ]
    return error, 8*(width*b+b)*np.log(4/beta)/(eps*n), indicator

In [549]:
errors = []
lengths = []
countt = 0
for i in range(200):
    error, length,indi = unboundedDomainShiftedMean(eps, beta, b, D)
    countt+=indi
    errors.append(error)
    lengths.append(length)
    
print(countt/200)
avgLength = sum(lengths)/200
errors.sort()
errorQuantile = errors[180]
print(avgLength/errorQuantile)

0.99
1.7412899207152888


In [550]:
print(avgLength/(sum(errors)/200))

4.0439662145015225


In [551]:
avgLength = sum(lengths)/200
errors.sort()
errorQuantile = errors[180]
print(avgLength/errorQuantile)

1.7412899207152888


In [588]:
def unboundedDomainMedian(eps, beta, b, D):
    n = len(D)
    intD = discretize(D,b)
    rad = unboundedDomainRadius(eps/2, beta/2, intD)
    clipped = clip(intD,-rad,rad)
    #define utility function
    u = np.zeros(n+2)#utility score
    l = np.zeros(n+3)#score changing point
    l[0] = -rad
    for i in range(n+2):
        if i<=n/2:
            u[i] = -n/2-1+i
            l[i+1]= clipped[i]
        u[int(n/2+1)] = 0
        l[int(n/2+2)] = clipped[int(n/2)]
        if i>n/2+1:
            u[i] = n/2+1-i
            l[i]= clipped[i-2]
            
    l[n+2] = rad
    #define u1
    factor = int(26/eps*np.log(4*rad/beta))+1
    u1 = np.zeros(n+2)
    u2 = np.zeros(n+2)
    for i in range(n+2):
        if i<=int(n/2):
            u1[i] = -abs(u[i]+factor)
            u2[i] = u[i]-factor
        else:
            u1[i] = u[i]-factor
            u2[i] = -abs(u[i]+factor)
            
    weight1 = np.zeros(n+2)
    weight2 = np.zeros(n+2)
    for i in range(n+2):
        weight1[i] = (l[i+1]-l[i])*math.pow(np.e, eps*u1[i]/8)
        weight2[i] = (l[i+1]-l[i])*math.pow(np.e, eps*u2[i]/8)
    weight1[int(n/2)] = math.pow(np.e, -factor)
    weight2[int(n/2)] = math.pow(np.e, -factor)
    print(np.argmax(weight1))
    print(np.argmax(weight2))
    
    totalWeight1 = sum(weight1)
    totalWeight2 = sum(weight2)
    print(totalWeight1,totalWeight2)

    weight1 = weight1/totalWeight1
    weight2 = weight2/totalWeight2

    i1 = np.random.choice(list(range(n+2)), p=weight1)
    i2 = np.random.choice(list(range(n+2)), p=weight2)
    if i1==int(n/2+1):
        x1 = l[i1]
    else:
        x1 = np.random.randint(l[i1], l[i1+1])
    if i2==int(n/2+1):
        x2 = l[i2]
    else:
        x2 = np.random.randint(l[i2], l[i2+1])
        
        
    return x1*b-b,x2*b+b   
    

In [589]:
x1,x2=unboundedDomainMedian(eps, beta, b, D)

702
1297
2.561525656511233 3.234875743999802


In [590]:
print(x1,x2)
print(D[int(n/2)])

41.6 57.800000000000004
50.31026735213851


In [555]:
def smoothMedian(eps, beta, D, GS,delta=0):
    #given D, first compute S(D)
    n = len(D)
    m = int(n/2)
    SS = 0
    if delta==0:
        alpha = eps/12
        for k in range(n+1):
            for t in range(k+2):
                if m+t>n-1:
                    rightx = GS
                else:
                    rightx = D[m+t]
                if m+t-k-1 <0:
                    leftx = -GS
                else:
                    leftx = D[m+t-k-1]
                temp = math.pow(math.e, -k*alpha)*(rightx-leftx)
                SS = max(SS,temp)
                
        print(SS)
        noisySS = SS*math.pow(math.e, 2*alpha/eps*LapNoise()+2*alpha/eps*math.log(2/beta))
        print(noisySS)        

        Q = D[int(n/2)]+12*SS/eps*np.random.standard_cauchy()
        print(Q)

        return Q, 12*noisySS/eps*math.tan((2-beta)*math.pi/4)
    else:
        alpha = eps/(4*math.log(1/delta))
        for k in range(n+1):
            for t in range(k+2):
                if m+t>n-1:
                    rightx = GS
                else:
                    rightx = D[m+t]
                if m+t-k-1 <0:
                    leftx = 0
                else:
                    leftx = D[m+t-k-1]
                temp = math.pow(math.e, -k*alpha)*(rightx-leftx)
                SS = max(SS,temp)
        print(SS)
        noisySS = SS*math.pow(math.e, 2*alpha/eps*LapNoise()+2*alpha/eps*math.log(2/beta))
        print(noisySS) 
        Q = D[int(n/2)]+4*SS/eps*LapNoise()

        return Q, 4*noisySS/eps*math.log(2/beta)
    

In [556]:
Q,l = smoothMedian(eps, beta, D, 1000000)

0.20196986723695756
0.3353520500228301
40.108442880674446


In [557]:
print(Q)
print(l)
print(D[int(n/2)])

40.108442880674446
51.13262167543172
50.31026735213851


In [558]:
Q,l = smoothMedian(eps, beta, D, 1000000,1e-6)

0.7059122341772299
0.7783771263444565


In [559]:
print(Q)
print(l)
print(Q-l)

50.419547082905446
9.327237913545202
41.09230916936024


In [560]:
def SVTquantile(eps, beta, D, tau, rad):
    TNoise = 2/eps*LapNoise()
    T = tau
    for i in range(2*rad+1):
        Ti = T-4/eps*np.log(2*(i+1)**2 * np.pi**2/(3*beta)) - 2/eps*np.log(4/beta)+TNoise
        Qi = count(D, -rad, -rad+i)+4/eps*LapNoise()
        if Qi>Ti:
            return -rad+i


In [576]:
def SVTMedian(eps, beta, b, D):
    n = len(D)
    intD = discretize(D,b)
    rad = int(unboundedDomainRadius(eps/2, beta/2, intD))
    print(rad)
    clipped = clip(intD,-rad,rad)
    factor = int(32/eps*math.log(4*(2*rad+1)**2 *math.pi**2/(3*beta))+16/eps*math.log(8/beta))+1
    ##Then run SVT, do not double
    ##find the n/2 +- factor quantile
    T1 = int(n/2)-1
    T2 = int(n/2)+factor
    XLeft = SVTquantile(eps/4, beta/4, clipped, T1, rad)
    print(XLeft)
    XRight = SVTquantile(eps/4, beta/4, clipped, T2, rad)
    print(XRight)
    
    return XLeft*b-b,XRight*b+b

In [595]:
XLeft,XRight = SVTMedian(eps, beta, b, D)

2048
402
590


In [596]:
print(XLeft)
print(XRight)

40.1
59.1


In [579]:
D[int(n/2)]

50.31026735213851

In [575]:
intD

array([-313., -144., -142., ..., 1113., 1127., 1183.])

In [593]:
32*math.log(1280*1183)

455.37357506854556

In [594]:
D[1400]-D[600]

20.912130275824524